## Use of Luong attention based encoder-decoder architecture model to perform the analysis of Hinglish text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Thu Dec  9 19:21:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Import the required dependencies**

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import pandas as pd
import unicodedata
import re
import numpy as np
import os
import io
import time
from string import digits
import pandas as pd

In [ ]:
import os
import gc
import time
import re
import unicodedata

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#Importing libraries
import tensorflow as tf
from tensorflow.keras import backend as K

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    ''' Preprocess the input text w applying lowercase, removing accents,
    creating a space between a word and the punctuation following it and
    replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    Input:
        - w: a string, input text
    Output:
        - a string, the cleaned text
    '''
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.strip()

    return w

In [ ]:
# Load the dataset: sentence in english

df=pd.read_excel("/content/drive/MyDrive/NLP/Hinglish50k.xlsx")

# Preprocess the input data
input_data=df["hindi_sentences"].apply(lambda x : preprocess_sentence(x)).tolist()

# Preprocess and include the end of sentence token to the target text
target_data=df["english_sentences"].apply(lambda x : preprocess_sentence(x)+ ' <eos>').tolist()

# Preprocess and include a start of setence token to the input text to the decoder, it is rigth shifted
target_input_data=df["english_sentences"].apply(lambda x : '<sos> '+ preprocess_sentence(x)).tolist()

print(input_data[:5])
print(target_data[:5])
print(target_input_data[:5])

['aur', 'eh .', 'men', 'oh .', 'aikya']
['and <eos>', 'eh . <eos>', 'men <eos>', 'oh . <eos>', 'iq <eos>']
['<sos> and', '<sos> eh .', '<sos> men', '<sos> oh .', '<sos> iq']


In [ ]:
MAX_VOCAB_SIZE = 20000
BATCH_SIZE = 32  # Batch size for training.
EPOCHS = 15  # Number of epochs

In [ ]:
# Create a tokenizer for the input texts and fit it to them
tokenizer_inputs = Tokenizer(num_words=MAX_VOCAB_SIZE, filters='')
tokenizer_inputs.fit_on_texts(input_data)
# Tokenize and transform input texts to sequence of integers
input_sequences = tokenizer_inputs.texts_to_sequences(input_data)
# Claculate the max length
input_max_len = max(len(s) for s in input_sequences)
print('Max Input Length: ', input_max_len)
# Show some example of tokenize sentences, useful to check the tokenization
print(input_data[1000])
print(input_sequences[1000])

Max Input Length:  24
yah dekho .
[17, 843, 1]


In [ ]:
tokenizer_outputs = Tokenizer(num_words=MAX_VOCAB_SIZE, filters='')
tokenizer_outputs.fit_on_texts(target_data)
tokenizer_outputs.fit_on_texts(target_input_data)
# T0okenize and transform output texts to sequence of integers
target_sequences = tokenizer_outputs.texts_to_sequences(target_data)
target_sequences_inputs = tokenizer_outputs.texts_to_sequences(target_input_data)

# determine maximum length output sequence
target_max_len = max(len(s) for s in target_sequences)
print('Max Target Length: ', target_max_len)

print(target_data[1000])
print(target_sequences[1000])
print(target_input_data[1000])
print(target_sequences_inputs[100])

Max Target Length:  20
watch this . <eos>
[735, 13, 1, 2]
<sos> watch this .
[3, 8089]


In [ ]:
# get the word to index mapping for input language
word2idx_inputs = tokenizer_inputs.word_index
print('Found %s unique input tokens.' % len(word2idx_inputs))

# get the word to index mapping for output language
word2idx_outputs = tokenizer_outputs.word_index
print('Found %s unique output tokens.' % len(word2idx_outputs))

# store number of output and input words for later
# remember to add 1 since indexing starts at 1
num_words_output = len(word2idx_outputs) + 1
num_words_inputs = len(word2idx_inputs) + 1

# map indexes back into real words
# so we can view the results
idx2word_inputs = {v:k for k, v in word2idx_inputs.items()}
idx2word_outputs = {v:k for k, v in word2idx_outputs.items()}

Found 24570 unique input tokens.
Found 21995 unique output tokens.


In [ ]:
# pad the input sequences
encoder_inputs = pad_sequences(input_sequences, maxlen=input_max_len, padding='post')
print("encoder_inputs.shape:", encoder_inputs.shape)
print("encoder_inputs[0]:", encoder_inputs[0])
# pad the decoder input sequences
decoder_inputs = pad_sequences(target_sequences_inputs, maxlen=target_max_len, padding='post')
print("decoder_inputs[0]:", decoder_inputs[0])
print("decoder_inputs.shape:", decoder_inputs.shape)
# pad the target output sequences
decoder_targets = pad_sequences(target_sequences, maxlen=target_max_len, padding='post')

encoder_inputs.shape: (49999, 24)
encoder_inputs[0]: [6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
decoder_inputs[0]: [3 8 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
decoder_inputs.shape: (49999, 20)


In [ ]:
# Define a dataset
dataset = tf.data.Dataset.from_tensor_slices(
    (encoder_inputs, decoder_inputs, decoder_targets))
dataset = dataset.shuffle(len(input_data)).batch(
    BATCH_SIZE, drop_remainder=True)

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(Encoder, self).__init__()
        self.hidden_dim = hidden_dim
        # Define the embedding layer
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        # Define the RNN layer, LSTM
        self.lstm = tf.keras.layers.LSTM(
            hidden_dim, return_sequences=True, return_state=True)

    def call(self, input_sequence, states):
        # Embed the input
        embed = self.embedding(input_sequence)
        # Call the LSTM unit
        output, state_h, state_c = self.lstm(embed, initial_state=states)

        return output, state_h, state_c

    def init_states(self, batch_size):
        # Return a all 0s initial states
        return (tf.zeros([batch_size, self.hidden_dim]),
                tf.zeros([batch_size, self.hidden_dim]))

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, attention_func):
        super(Decoder, self).__init__()
        self.attention = LuongAttention(hidden_dim, attention_func)
        self.hidden_dim = hidden_dim
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(
            hidden_dim, return_sequences=True, return_state=True)
        self.wc = tf.keras.layers.Dense(hidden_dim, activation='tanh')
        self.ws = tf.keras.layers.Dense(vocab_size)

    def call(self, input_sequence, state, encoder_output):
        # Remember that the input to the decoder
        # is now a batch of one-word sequences,
        # which means that its shape is (batch_size, 1)
        embed = self.embedding(input_sequence)

        # Therefore, the lstm_out has shape (batch_size, 1, hidden_dim)
        lstm_out, state_h, state_c = self.lstm(embed, initial_state=state)

        # Use self.attention to compute the context and alignment vectors
        # context vector's shape: (batch_size, 1, hidden_dim)
        # alignment vector's shape: (batch_size, 1, source_length)
        context, alignment = self.attention(lstm_out, encoder_output)

        # Combine the context vector and the LSTM output
        # Before combined, both have shape of (batch_size, 1, hidden_dim),
        # so let's squeeze the axis 1 first
        # After combined, it will have shape of (batch_size, 2 * hidden_dim)
        lstm_out = tf.concat(
            [tf.squeeze(context, 1), tf.squeeze(lstm_out, 1)], 1)

        # lstm_out now has shape (batch_size, hidden_dim)
        lstm_out = self.wc(lstm_out)

        # Finally, it is converted back to vocabulary space: (batch_size, vocab_size)
        logits = self.ws(lstm_out)

        return logits, state_h, state_c, alignment

In [ ]:
class LuongAttention(tf.keras.Model):
    def __init__(self, rnn_size, attention_func):
        super(LuongAttention, self).__init__()
        self.attention_func = attention_func

        if attention_func not in ['dot', 'general', 'concat']:
            raise ValueError(
                'Attention score must be either dot, general or concat.')

        if attention_func == 'general':
            # General score function
            self.wa = tf.keras.layers.Dense(rnn_size)
        elif attention_func == 'concat':
            # Concat score function
            self.wa = tf.keras.layers.Dense(rnn_size, activation='tanh')
            self.va = tf.keras.layers.Dense(1)

    def call(self, decoder_output, encoder_output):
        if self.attention_func == 'dot':
            # Dot score function: decoder_output (dot) encoder_output
            # decoder_output has shape: (batch_size, 1, rnn_size)
            # encoder_output has shape: (batch_size, max_len, rnn_size)
            # => score has shape: (batch_size, 1, max_len)
            score = tf.matmul(decoder_output, encoder_output, transpose_b=True) # (batch_size, 1, max_len)
        elif self.attention_func == 'general':
            # General score function: decoder_output (dot) (Wa (dot) encoder_output)
            # decoder_output has shape: (batch_size, 1, rnn_size)
            # encoder_output has shape: (batch_size, max_len, rnn_size)
            # => score has shape: (batch_size, 1, max_len)
            score = tf.matmul(decoder_output, self.wa(
                encoder_output), transpose_b=True) #(batch_size, 1, max_len)
        elif self.attention_func == 'concat':
            # Concat score function: va (dot) tanh(Wa (dot) concat(decoder_output + encoder_output))
            # Decoder output must be broadcasted to encoder output's shape first
            decoder_output = tf.tile(
                decoder_output, [1, encoder_output.shape[1], 1]) #shape (batch size, max len,hidden_dim)

            # Concat => Wa => va
            # (batch_size, max_len, 2 * rnn_size) => (batch_size, max_len, rnn_size) => (batch_size, max_len, 1)
            score = self.va(
                self.wa(tf.concat((decoder_output, encoder_output), axis=-1))) # (batch_size, max len, 1)

            # Transpose score vector to have the same shape as other two above
            # (batch_size, max_len, 1) => (batch_size, 1, max_len)
            score = tf.transpose(score, [0, 2, 1]) #(batch_size, 1, max_len)

        # alignment a_t = softmax(score)
        alignment = tf.keras.activations.softmax(score, axis=-1) #(batch_size, 1, max_len)

        # context vector c_t is the weighted average sum of encoder output
        context = tf.matmul(alignment, encoder_output) # (batch_size, 1, hidden_dim)

        return context, alignment

In [ ]:
ATTENTION_FUNC='general'

In [ ]:
EMBEDDING_DIM = 128
HIDDEN_DIM=1024 #512

In [ ]:
#Set the length of the input and output vocabulary
num_words_inputs = len(word2idx_inputs) + 1
num_words_output = len(word2idx_outputs) + 1
#Create the encoder
encoder = Encoder(num_words_inputs, EMBEDDING_DIM, HIDDEN_DIM)
decoder = Decoder(num_words_output, EMBEDDING_DIM, HIDDEN_DIM, ATTENTION_FUNC)

# Call the encoder and then the decoder
initial_state = encoder.init_states(1)
encoder_outputs = encoder(tf.constant([[1]]), initial_state)
decoder_outputs = decoder(tf.constant(
    [[1]]), encoder_outputs[1:], encoder_outputs[0])

In [ ]:
@tf.function
def train_step(input_seq, target_seq_in, target_seq_out, en_initial_states, optimizer):
    ''' A training step, train a batch of the data and return the loss value reached
        Input:
        - input_seq: array of integers, shape [batch_size, max_seq_len, embedding dim].
            the input sequence
        - target_seq_out: array of integers, shape [batch_size, max_seq_len, embedding dim].
            the target seq, our target sequence
        - target_seq_in: array of integers, shape [batch_size, max_seq_len, embedding dim].
            the input sequence to the decoder, we use Teacher Forcing
        - en_initial_states: tuple of arrays of shape [batch_size, hidden_dim].
            the initial state of the encoder
        - optimizer: a tf.keras.optimizers.
        Output:
        - loss: loss value

    '''
    loss = 0.
    acc = 0.
    logits = None

    with tf.GradientTape() as tape:
        en_outputs = encoder(input_seq, en_initial_states)
        en_states = en_outputs[1:]
        de_state_h, de_state_c = en_states

        # We need to create a loop to iterate through the target sequences
        for i in range(target_seq_out.shape[1]):
            # Input to the decoder must have shape of (batch_size, length)
            # so we need to expand one dimension
            decoder_in = tf.expand_dims(target_seq_in[:, i], 1)
            logit, de_state_h, de_state_c, _ = decoder(
                decoder_in, (de_state_h, de_state_c), en_outputs[0])

            # The loss is now accumulated through the whole batch
            loss += loss_func(target_seq_out[:, i], logit)
            # Store the logits to calculate the accuracy
            logit = K.expand_dims(logit, axis=1)
            if logits is None:
                logits = logit
            else:
                logits = K.concatenate((logits,logit), axis=1)
        # Calculate the accuracy for the batch data
        acc = accuracy_fn(target_seq_out, logits)
    # Update the parameters and the optimizer
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return loss / target_seq_out.shape[1], acc

In [ ]:
# Create the main train function
def main_train(encoder, decoder, dataset, n_epochs, batch_size, optimizer, checkpoint, checkpoint_prefix):

    losses = []
    accuracies = []

    for e in range(n_epochs):
        # Get the initial time
        start = time.time()
        # Get the initial state for the encoder
        en_initial_states = encoder.init_states(batch_size)
        # For every batch data
        for batch, (input_seq, target_seq_in, target_seq_out) in enumerate(dataset.take(-1)):
            # Train and get the loss value
            loss, accuracy = train_step(input_seq, target_seq_in, target_seq_out, en_initial_states, optimizer)

            if batch % 100 == 0:
                # Store the loss and accuracy values
                losses.append(loss)
                accuracies.append(accuracy)
                print('Epoch {} Batch {} Loss {:.4f} Acc:{:.4f}'.format(e + 1, batch, loss.numpy(), accuracy.numpy()))

        # saving (checkpoint) the model every 2 epochs
        if (e + 1) % 2 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

        print('Time taken for 1 epoch {:.4f} sec\n'.format(time.time() - start))

    return losses, accuracies

In [ ]:
def loss_func(targets, logits):
    crossentropy = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True)
    # Mask padding values, they do not have to compute for loss
    mask = tf.math.logical_not(tf.math.equal(targets, 0))
    mask = tf.cast(mask, dtype=tf.int64)
    # Calculate the loss value
    loss = crossentropy(targets, logits, sample_weight=mask)

    return loss

def accuracy_fn(y_true, y_pred):
    # y_pred shape is batch_size, seq length, vocab size
    # y_true shape is batch_size, seq length
    pred_values = K.cast(K.argmax(y_pred, axis=-1), dtype='int32')
    correct = K.cast(K.equal(y_true, pred_values), dtype='float32')

    # 0 is padding, don't include those
    mask = K.cast(K.greater(y_true, 0), dtype='float32')
    n_correct = K.sum(mask * correct)
    n_total = K.sum(mask)

    return n_correct / n_total

In [ ]:
# Create an Adam optimizer and clips gradients by norm
optimizer = tf.keras.optimizers.Adam(clipnorm=5.0)
# Create a checkpoint object to save the model
checkpoint_dir = './training_ckpt_seq2seq_att'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

losses, accuracies = main_train(encoder, decoder, dataset, EPOCHS, BATCH_SIZE, optimizer, checkpoint, checkpoint_prefix)

Epoch 1 Batch 0 Loss 3.7963 Acc:0.0000
Epoch 1 Batch 100 Loss 2.6536 Acc:0.1296
Epoch 1 Batch 200 Loss 2.3875 Acc:0.1912
Epoch 1 Batch 300 Loss 2.5453 Acc:0.1577
Epoch 1 Batch 400 Loss 2.1477 Acc:0.1966
Epoch 1 Batch 500 Loss 2.4088 Acc:0.2261
Epoch 1 Batch 600 Loss 2.2102 Acc:0.2313
Epoch 1 Batch 700 Loss 2.5440 Acc:0.2407
Epoch 1 Batch 800 Loss 1.9171 Acc:0.2562
Epoch 1 Batch 900 Loss 2.0168 Acc:0.2648
Epoch 1 Batch 1000 Loss 2.1904 Acc:0.2453
Epoch 1 Batch 1100 Loss 2.0152 Acc:0.2857
Epoch 1 Batch 1200 Loss 1.7899 Acc:0.2911
Epoch 1 Batch 1300 Loss 2.0435 Acc:0.2530
Epoch 1 Batch 1400 Loss 2.3464 Acc:0.2828
Epoch 1 Batch 1500 Loss 1.9935 Acc:0.3098
Time taken for 1 epoch 627.8108 sec

Epoch 2 Batch 0 Loss 1.8579 Acc:0.3095
Epoch 2 Batch 100 Loss 1.9261 Acc:0.2717
Epoch 2 Batch 200 Loss 1.7604 Acc:0.3162
Epoch 2 Batch 300 Loss 2.1833 Acc:0.3014
Epoch 2 Batch 400 Loss 1.5512 Acc:0.3511
Epoch 2 Batch 500 Loss 1.9463 Acc:0.3109
Epoch 2 Batch 600 Loss 1.9646 Acc:0.3248
Epoch 2 Batch 700 

**---------------------------------------------------------**

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5))
# plot some data
ax1.plot(losses, label='loss')
#plt.plot(results.history['val_loss'], label='val_loss')
ax1.set_title('Training Loss')
ax1.legend()
# accuracies
ax2.plot(accuracies, label='acc')
#plt.plot(results.history['val_accuracy_fn'], label='val_acc')
ax2.set_title('Training Accuracy')
ax2.legend()
plt.show()

In [ ]:
# Create an Adam optimizer and clips gradients by norm
optimizer = tf.keras.optimizers.Adam(clipnorm=5.0)
# Create a checkpoint object to save the model
checkpoint_dir = './training_ckpt_seq2seq_att'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
def predict_seq2seq_att(input_text, input_max_len, tokenizer_inputs, word2idx_outputs, idx2word_outputs):
    if input_text is None:
        input_text = input_data[np.random.choice(len(input_data))]
    print(input_text)
    # Tokenize the input text
    input_seq = tokenizer_inputs.texts_to_sequences([input_text])
    # Pad the sentence
    input_seq = pad_sequences(input_seq, maxlen=input_max_len, padding='post')
    # Get the encoder initial states
    en_initial_states = encoder.init_states(1)
    # Get the encoder outputs or hidden states
    en_outputs = encoder(tf.constant(input_seq), en_initial_states)
    # Set the decoder input to the sos token
    de_input = tf.constant([[word2idx_outputs['<sos>']]])
    # Set the initial hidden states of the decoder to the hidden states of the encoder
    de_state_h, de_state_c = en_outputs[1:]

    out_words = []
    alignments = []

    while True:
        # Get the decoder with attention output
        de_output, de_state_h, de_state_c, alignment = decoder(
            de_input, (de_state_h, de_state_c), en_outputs[0])
        de_input = tf.expand_dims(tf.argmax(de_output, -1), 0)
        # Detokenize the output
        out_words.append(idx2word_outputs[de_input.numpy()[0][0]])
        # Save the aligment matrix
        alignments.append(alignment.numpy())

        if out_words[-1] == '<eos>' or len(out_words) >= 20:
            break
    # Join the output words
    print(' '.join(out_words))
    return np.array(alignments), input_text.split(' '), out_words

**Emoji analysis**

In [ ]:
!pip install emot

     |████████████████████████████████| 61 kB 16 kB/s 


In [ ]:
import re
import pickle

with open('/content/drive/MyDrive/NLP/Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emojis_to_word(text):
    for emot in Emoji_Dict:
        text = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), text)
    return text

**Short forms / SMS Slang**

In [ ]:
messages = { "AFAIK" : "As Far As I Know",
"AFK" : "Away From Keyboard",
"ASAP" : "As Soon As Possible",
"ATK" : "At The Keyboard",
"ATM" : "At The Moment",
"A3" : "Anytime, Anywhere, Anyplace",
"BAK" : "Back At Keyboard",
"BBL" : "Be Back Later",
"BBS" : "Be Back Soon",
"BFN" : "Bye For Now",
"B4N" : "Bye For Now",
"BRB" : "Be Right Back",
"BRT" : "Be Right There",
"BTW" : "By The Way",
"B4" : "Before",
"B4N" : "Bye For Now",
"CU" : "See You",
"CUL8R" : "See You Later",
"CYA" : "See You",
"FAQ" : "Frequently Asked Questions",
"FC" : "Fingers Crossed",
"FWIW" : "For What It's Worth",
"FYI" : "For Your Information",
"GAL" : "Get A Life",
"GG" : "Good Game",
"GN" : "Good Night",
"GMTA":"Great Minds Think Alike",
"GR8" : "Great!",
"G9" : "Genius",
"IC" : "I See",
"ICQ" : "I Seek you",
"ILU": "I Love You",
"ILY": "I Love You",
"IMHO" : "In My Honest/Humble Opinion",
"IMO" : "In My Opinion",
"IOW" : "In Other Words",
"IRL" : "In Real Life",
"KISS" : "Keep It Simple, Stupid",
"LDR" : "Long Distance Relationship",
"LMAO" : "Laugh My A.. Off",
"LOL" : "Laughing Out Loud",
"LTNS" : "Long Time No See",
"L8R" : "Later",
"MTE" : "My Thoughts Exactly",
"M8" : "Mate",
"NRN" : "No Reply Necessary",
"OIC" : "Oh I See",
"PITA" : "Pain In The A",
"PRT" : "Party",
"PRW" : "Parents Are Watching",
"QPSA?" : 	"Que Pasa?",
"ROFL" : "Rolling On The Floor Laughing",
"ROFLOL" : "Rolling On The Floor Laughing Out Loud",
"ROTFLMAO" : "Rolling On The Floor Laughing My A.. Off",
"SK8" : "Skate",
"STATS" : "Your sex and age",
"ASL" : "Age, Sex, Location",
"THX" : "Thank You",
"TTFN" : "Ta-Ta For Now!",
"TTYL" : "Talk To You Later",
"U" : "You",
"U2" : "You Too",
"U4E" : "Yours For Ever",
"WB" : "Welcome Back",
"WTF": "What The F...",
"WTG" : "Way To Go!",
"WUF" : "Where Are You From?",
"W8"  : "Wait..."
}

In [ ]:
def translates(sent):
  new_sent = ""
  words = sent.split(" ")
  for word in words:
    if word.upper() in messages.keys():
      new_sent+= messages[word.upper()]
    else:
      new_sent+=word
    new_sent+=" "
  return new_sent

**UI - Flask APP**

In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 1.9 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=e65f1ada28909cd30da29de2c2ab83ccd5f215ccd98d87906458cf3f84730ee8
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
!ngrok authtoken "21zLoyqOXDMBV3J0A0UA2TUIJKt_3unqjxyfWKpbXu6A629ke"

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!pip install codeswitch

     |████████████████████████████████| 3.3 MB 2.0 MB/s 
     |████████████████████████████████| 61 kB 437 kB/s 
     |████████████████████████████████| 596 kB 43.3 MB/s 
     |████████████████████████████████| 895 kB 38.3 MB/s 
     |████████████████████████████████| 3.3 MB 37.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install translator

In [ ]:
!pip install translate

In [ ]:
!pip install nltk

In [ ]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
from translate import Translator
from codeswitch.codeswitch import NER
from codeswitch.codeswitch import POS
from codeswitch.codeswitch import LanguageIdentification

from nltk.translate.bleu_score import sentence_bleu
app = Flask(__name__, template_folder='/content/drive/MyDrive/NLP/templates',
            static_folder='/content/drive/MyDrive/NLP/static/images')
run_with_ngrok(app)


@app.route("/")
def home():
  return render_template("main.html")

@app.route("/analytics")
def graph():
    return render_template("displayGraph.html")

@app.route("/mainFeatures")
def feature():
    return render_template("mainFeatures.html")

@app.route("/translate")
def translateText():
    return render_template("translate.html")

@app.route("/langResult", methods=['POST'])
def trans():
  res_source = ""
  res_dest = ""
  test_sent = " "
  sent = request.form["text"]
  test_sent = convert_emojis_to_word(sent)
  test_sent = translates(sent)
  alignments, source, prediction = predict_seq2seq_att(sent, input_max_len, tokenizer_inputs,
                                                     word2idx_outputs, idx2word_outputs)
  for x in source:
    res_source +=x
  for y in prediction:
    res_dest+=y
    res_dest+=" "

  return test_sent+ " ** " + res_dest

@app.route("/shortform")
def show():
    return render_template("emoShortForm.html")

@app.route("/esf", methods=["POST"])
def esf():
    my_sent = request.form['text']
    user_ip = my_sent
    my_sent = convert_emojis_to_word(my_sent)
    my_sent = translates(my_sent)
    return render_template("smsShow.html", text=user_ip, answer=my_sent)

@app.route("/check")
def check():
    return render_template("Bleu.html")

@app.route("/calculate", methods=['POST'])
def calculate():
    text = request.form['text']
    original_list = [text.split(" ")]
    translator= Translator(from_lang="hindi",to_lang="english")
    translation = translator.translate(text)
    destination_list = translation.split(" ")
    score = sentence_bleu(original_list, destination_list)
    return render_template("result.html", score=score, inter=translation, text=text)

@app.route("/bot")
def bot():
    return render_template("bot.html")

@app.route("/ner")
def ner():
    return render_template("ner.html")

@app.route("/nerresult", methods=["POST"])
def nerresult():
    ner = NER('hin-eng')
    txt = request.form['text']
    result = ner.tag(txt)
    return render_template("nerResult.html", result=result)

@app.route("/language")
def postag():
    return render_template("pos.html")


@app.route("/posresult", methods=["POST"])
def posres():
    text = request.form['text']
    pos = POS('hin-eng')
    result = pos.tag(text)
    return render_template("posResult.html", result=result)

@app.route("/identify")
def ident():
    return render_template("langIdentify.html")

@app.route("/langresult", methods=["POST"])
def langres():
    text = request.form['text']
    lid = LanguageIdentification('hin-eng')
    result = lid.identify(text)
    return render_template("langResult.html", result=result)

app.run()



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://1543-34-80-4-208.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Dec/2021 19:55:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2021 19:55:18] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2021 19:55:21] "GET /translate HTTP/1.1" 200 -


mujhe morning mai ghoomna bahut pasand hai


127.0.0.1 - - [09/Dec/2021 19:55:28] "POST /langResult HTTP/1.1" 200 -


i m going to see a lot of a lot of i . <eos>
